# Notebook for launching Oracle Solaris on OCI

Notebook for Working with the OCI Python API to launch an Oracle Solaris from the Marketplace image. This notebook contains the basic steps needed to find the image in the Marketplace, check if you've agreed to the license and if not how to do this, and finally with all this information how to launch a new instance of Oracle Solaris in OCI using this Marketplace image.

First we need to import the needed Python libraries, load the configuration files, and initialize the Python client objects we're going to need.

Importing the libraries:

In [1]:
import oci
from configparser import ConfigParser, DEFAULTSECT
from terminaltables import AsciiTable
from os.path import expanduser

Defining variables:

In [2]:
home = expanduser("~")
config_file = f'{home}/.oci/config' # The location of your OCI config file
rc_file = f'{home}/.oci/oci_compute_rc' # The location of your rc file
profile = 'DEFAULT' # The profile you want to load from the rc file
market_image_name = 'Oracle Solaris 11.4' # The name of the Oracle Solaris image in the OCI Marketplace

The `config` file is the same file as you would use when using the OCI CLI. It contains information like your tenancy, user, and region information and will be used to initialize the Python clients used to connect to OCI.

The `oci_compute_rc` file is a file you will have to create yourself and it hold the information needed to define the instance you want to create. It contains things like the the Compartment ID, Availability Domain, and the VCN name so the instance is created in and connected to the right places and elements in OCI. It looks like this:

```
[DEFAULT]
# Global defaults for oci-compute
compartment-id = ocid1.compartment.oc1..xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
shape = VM.Standard2.1
availability-domain = <AD-name>
vcn-name = <VCN-name>
subnet-name = <Subnet-name>
ssh-authorized-keys-file = /<directory>/<ssh-guest-key-name>
```

If you don't know the values of these elements you can find them in the Oracle Cloud Console.

Now we load the rc file:

In [3]:
my_instance_config = ConfigParser(interpolation=None)

print(my_instance_config.read(rc_file))

for key in my_instance_config[profile]:
    print(f'{key:>25}: {my_instance_config[profile][key]}')

['~/.oci/oci_compute_rc']
           compartment-id: ocid1.compartment.oc1..xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
                    shape: VM.Standard2.1
      availability-domain: AD-1
                 vcn-name: network-01
              subnet-name: subnet-01
 ssh-authorized-keys-file: ~/.oci/ssh-guest-key-20221028.key.pub


And the config file using the `oci.config.from_file` function to load it into the right object:

In [4]:
my_account_config = oci.config.from_file(config_file, profile)

for key in my_account_config:
    print(f'{key:>25}: {my_account_config[key]}')

             log_requests: False
    additional_user_agent: 
              pass_phrase: None
                     user: ocid1.user.oc1..xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
              fingerprint: 94:79:e9:17:44:23:8e:67:a5:d7:f2:58:9f:aa:db:61
                 key_file: ~/.oci/oci_api_key.pem
                  tenancy: ocid1.tenancy.oc1..xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
                   region: eu-frankfurt-1


And now we initialize the Python client objects using the config file information:

In [5]:
my_compute_client = oci.core.ComputeClient(my_account_config)
my_identity_client = oci.identity.IdentityClient(my_account_config)
my_virtual_network_client = oci.core.VirtualNetworkClient(my_account_config)
my_marketplace_client = oci.marketplace.MarketplaceClient(my_account_config)

At this point we are ready for the first steps.

## Working with the Marketplace

The first thing we'll need to do is find the Oracle Solaris listing in the Marketplace. This will allow us to find the list of available Oracle Solaris versions also known as packages. 

### Getting the details of the Oracle Solaris 11.4 images

Using the Marketplace client we load the list of Marketplace listings and search for the listing named `Oracle Solaris 11.4`. This will give us a list with one entry, and we pull this entry out, store it in `listing`, and print it to show you what it looks like. 

> Note: The `list_listings()` fucntion normally only loads a subset of the total so we use the `oci.pagination.list_call_get_all_results()` function to load all the results in one go.

Once we have the listing information we can pull the list of packages associated with Oracle Solaris. Each package is a different version of Oracle Solaris where we order it with newest version first. So if we pull the first package in the list we'll automatically get the most recent version.

Then using the information about this package we fetch the full set of data associated with the package.

In [6]:
response = oci.pagination.list_call_get_all_results(my_marketplace_client.list_listings, pricing=['FREE'])
listings = []

# Searching for Oracle Solaris 11.4 in the listings
for listing in response.data:
    if market_image_name in listing.name:
        listings.append(listing)

listing = listings[0]
print(f'Listing:\n{listing}')

packages = my_marketplace_client.list_packages(listing.id, sort_by='TIMERELEASED', sort_order='DESC').data
package = packages[0]

package = my_marketplace_client.get_package(package.listing_id, package.package_version).data
print(f'\nThe lastest package:\n{package}')

Listing:
{
  "categories": [
    "Operating Systems"
  ],
  "compatible_architectures": [
    "X86"
  ],
  "icon": {
    "content_url": "https://objectstorage.eu-frankfurt-1.oraclecloud.com/p/cEyHNruC431YvypjHWa3pYeU8ATum3oOTS_vMSr-a2fIiv1Vucl-fByudYY9D4ah/n/marketplaceprod/b/pacman-prod-consumer-content/o/136471195.png",
    "file_extension": "png",
    "mime_type": "image/png",
    "name": ""
  },
  "id": "61750333",
  "is_featured": false,
  "is_rover_exportable": false,
  "listing_type": "PARTNER",
  "name": "Oracle Solaris 11.4",
  "package_type": "IMAGE",
  "pricing_types": [
    "FREE"
  ],
  "publisher": {
    "description": "With more than 380,000 customers\u2014including 100 of the Fortune 100\u2014and with deployments across a wide variety of industries in more than 145 countries around the globe, Oracle offers an optimized and fully integrated stack of business hardware and software systems. Oracle engineers hardware and software to work together in the cloud and in your da

You can actually use this to also list all the listings in the Marketplace, this is a list of the free listings:

In [7]:
response = oci.pagination.list_call_get_all_results(my_marketplace_client.list_listings, pricing=['FREE'])
listings = set()
for listing in response.data:
    listings.add((listing.publisher.name, listing.name))

table = AsciiTable([('Publisher', 'Name')] + sorted(listings))
table.title = 'Free Marketplace images'
print(table.table)

+Free Marketplace images----------+-----------------------------------------------------------------------------+
| Publisher                       | Name                                                                        |
+---------------------------------+-----------------------------------------------------------------------------+
| AlmaLinux Foundation            | AlmaLinux OS 8 (AArch64)                                                    |
| AlmaLinux Foundation            | AlmaLinux OS 8 (x86_64)                                                     |
| AlmaLinux Foundation            | AlmaLinux OS 9 (AArch64)                                                    |
| AlmaLinux Foundation            | AlmaLinux OS 9 (x86_64)                                                     |
| Ampere Computing LLC            | ONNXRT - Ampere® Optimized Framework - Ubuntu 20.04                         |
| Ampere Computing LLC            | PyTorch - Ampere® Optimized Framework - Ubuntu 20.04

### Checking for a license agreement for this package

Now we have the package information we need we can use it to check if you've already accepted the license. To do this you first need to use the Marketplace client to fetch the list of agreements you have in place against this package, plus we get the list of accepted agreements you have within this compartment, against the package. 

Then we check if the agreement we fetched is in the list of accepted agreements. If so you're ready for the next step, if not then you need to accept the agreement first.

The following code does this and prints out the result:

In [8]:
agreements = my_marketplace_client.list_agreements(package.listing_id, package.version).data
print(f'The list of agreements:\n{agreements}\n')

accepted_agreements = my_marketplace_client.list_accepted_agreements(my_instance_config[profile]['compartment-id'], 
                                                                     listing_id=package.listing_id, 
                                                                     package_version=package.version).data
print(f'The list of accepted agreements:\n{accepted_agreements}\n')

if accepted_agreements:
    print(f'The details of the current accepted agreement:\n{accepted_agreements}\n')
else:
    print('There is currently no agreement in place\n')

not_accepted = []
for agreement in agreements:
    agreement_match = [accepted_agreement for accepted_agreement in accepted_agreements if agreement.id == accepted_agreement.agreement_id]
    if not agreement_match:
        not_accepted.append(agreement)
        print(f'The agreement was not accepted yet. The terms of the agreement:\n{agreement}\n')
    else:
        print(f'The agreement was accepted on: {agreement_match[0].time_accepted}\n')

The list of agreements:
[{
  "author": "ORACLE",
  "content_url": "https://objectstorage.eu-frankfurt-1.oraclecloud.com/p/cEyHNruC431YvypjHWa3pYeU8ATum3oOTS_vMSr-a2fIiv1Vucl-fByudYY9D4ah/n/marketplaceprod/b/pacman-prod-consumer-content/o/58993511.html",
  "id": "58993510",
  "prompt": "I have reviewed and accept the <link>Terms of Use for Oracle Solaris on the Oracle Cloud Infrastructure Marketplace</link>"
}]

The list of accepted agreements:
[{
  "agreement_id": "58993510",
  "compartment_id": "ocid1.compartment.oc1..xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "display_name": "marketplaceacceptedagreement20221031181133",
  "id": "ocid1.marketplaceacceptedagreement.oc1..xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "listing_id": "61750333",
  "package_version": "11.4.50",
  "time_accepted": "2022-10-31T18:11:33.995000+00:00"
}]

The details of the current accepted agreement:
[{
  "agreement_id": "58993510",
  "compartment_id": "ocid1.compartment

### To accept the agreement for this package

If the agreement was not yet accepted, this following code will get the agreement details, then create an accepted agreement details object, and then use this object to accept the agreement:

In [9]:
if not_accepted:
    for agreement in not_accepted:
        agreement_detail = my_marketplace_client.get_agreement(package.listing_id, package.version, agreement.id).data
        print(f'The details of the agreement before accepting the license:\n{agreement_detail}\n')
        
        accepted_agreement_details = oci.marketplace.models.CreateAcceptedAgreementDetails(
            agreement_id=agreement.id,
            compartment_id=my_instance_config[profile]['compartment-id'],
            listing_id=package.listing_id,
            package_version=package.version,
            signature=agreement_detail.signature)
        print(f'The details of the agreement acceptment statement:\n{accepted_agreement_details}\n')
        my_marketplace_client.create_accepted_agreement(accepted_agreement_details)

else:
    print('Agreements already accepted')

Agreements already accepted


## Provisioning the Oracle Solaris 11.4 image from the Marketplace

Now you have all the information about the Marketplace image you need and you've checked if the license has been accepted, you can provision an instance.

### Defining the new instance

To create the instance we need to define a set of Python variables and objects:

* Display Name
* Compartment ID
* Availability Domain Name
* Shape Name
* VNIC Details Object
* Image Details Object
* Metadata Object — That holds info like where the SSH key is located

We know the Compartment ID and the Shape as they were defined in the RC file.

First, we choose a Display Name for the new instance:

In [10]:
new_instance_name = 'provisioning_with_python'

Then we check the Availability Domain. This way you can define a very simple Availability Domain name, like `AD1` and then use this to find the real Availability Domain name:

In [11]:
availability_domains = oci.pagination.list_call_get_all_results(my_identity_client.list_availability_domains, my_instance_config[profile]['compartment-id']).data
ad_match = [ad for ad in availability_domains if my_instance_config[profile]['availability-domain'].upper() in ad.name]
if ad_match:
    print(f'Found availability domain name is {ad_match[0].name}')
else:
    print(f"No AD found matching \"{my_instance_config[profile]['availability-domain']}\"")

Found availability domain name is ruWb:EU-FRANKFURT-1-AD-1


Next we check if the VCN name and Subnet name given in the RC file are valid and exist in the compartment we've chosen, and using the Subnet information we create a VNIC object we need for the instance:

In [12]:
vcns = my_virtual_network_client.list_vcns(my_instance_config[profile]['compartment-id'], display_name=my_instance_config[profile]['vcn-name']).data
if not vcns:
    print('No matching VCN for "{}"'.format(my_instance_config[profile]['vcn-name']))
else:
    vcn = vcns[0]
    print(f'Found VCN name is {vcn.display_name}')
    subnets = my_virtual_network_client.list_subnets(my_instance_config[profile]['compartment-id'], 
                                                     vcn_id=vcn.id, 
                                                     display_name=my_instance_config[profile]['subnet-name']).data
    if not subnets:
        print('No matching subnet for "{}"'.format(my_instance_config[profile]['subnet-name']))
    else:
        subnet = subnets[0]
        print(f'Found Subnet name is {subnet.display_name}')
        
create_vnic_details = oci.core.models.CreateVnicDetails(subnet_id=subnet.id)
print(f'\nThe VNIC details are:\n{create_vnic_details}')

Found VCN name is network-01
Found Subnet name is subnet-01

The VNIC details are:
{
  "assign_private_dns_record": null,
  "assign_public_ip": null,
  "defined_tags": null,
  "display_name": null,
  "freeform_tags": null,
  "hostname_label": null,
  "nsg_ids": null,
  "private_ip": null,
  "skip_source_dest_check": null,
  "subnet_id": "ocid1.subnet.oc1.eu-frankfurt-1.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "vlan_id": null
}


Now we take the package infromation from the OCI Marketplace and use it to find the information about this image we want to use on the OCI Compute side, so it can use it to provision the instance. 

We use the `get_app_catalog_listing_resource_version()` function and the Marketplace package information to locate the listing on the OCI Compute side. We can then use the `get_image()` function to create the Image Object, which is then used by the `InstanceSourceViaImageDetails()` function to create the Image Details Obejct needed:

In [13]:
app_catalog_listing_resource_version = my_compute_client.get_app_catalog_listing_resource_version(
            package.app_catalog_listing_id,
            package.app_catalog_listing_resource_version).data
print(f'The App Catalog listing for the package:\n{app_catalog_listing_resource_version}')
    
image = my_compute_client.get_image(app_catalog_listing_resource_version.listing_resource_id).data
print(f'\nThe image object for the package:\n{image}')

instance_source_via_image_details = oci.core.models.InstanceSourceViaImageDetails(image_id=image.id)
print(f'\nThe image details object:\n{instance_source_via_image_details}')

The App Catalog listing for the package:
{
  "accessible_ports": [],
  "allowed_actions": [
    "SNAPSHOT"
  ],
  "available_regions": [
    "af-johannesburg-1",
    "ap-chuncheon-1",
    "ap-hyderabad-1",
    "ap-melbourne-1",
    "ap-mumbai-1",
    "ap-osaka-1",
    "ap-seoul-1",
    "ap-singapore-1",
    "ap-sydney-1",
    "ap-tokyo-1",
    "ca-montreal-1",
    "ca-toronto-1",
    "eu-amsterdam-1",
    "eu-frankfurt-1",
    "eu-madrid-1",
    "eu-marseille-1",
    "eu-milan-1",
    "eu-paris-1",
    "eu-stockholm-1",
    "eu-zurich-1",
    "il-jerusalem-1",
    "me-abudhabi-1",
    "me-dubai-1",
    "me-jeddah-1",
    "mx-queretaro-1",
    "sa-santiago-1",
    "sa-saopaulo-1",
    "sa-vinhedo-1",
    "uk-cardiff-1",
    "uk-london-1",
    "us-ashburn-1",
    "us-chicago-1",
    "us-phoenix-1",
    "us-sanjose-1"
  ],
  "compatible_shapes": [
    "VM.Standard.B1.2",
    "BM.DenseIO2.52",
    "VM.DenseIO1.16",
    "VM.Standard.B1.1",
    "BM.DenseIO1.36",
    "VM.Standard.B1.4",
    "

And define the Metadata Object:

In [14]:
metadata = {}
with open(my_instance_config[profile]['ssh-authorized-keys-file']) as ssh_authorized_keys:
    metadata['ssh_authorized_keys'] = ssh_authorized_keys.read()
print(metadata)

{'ssh_authorized_keys': 'ssh-rsa xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx ssh-key-2022-10-28'}


### Launching the instance

We finally get to launching the instance. 

We use the `LaunchInstanceDetails()` function to bundle up all the information needed to launch the instance and then launch it.

However because it can take a bit of time before the instance is fully launched OCI provides a `launch_instance_and_wait_for_state()` function that you can use together with a `wait_for_states` value to launch the instance and it will wait with returning until the instance is running.

In [15]:
launch_instance_details = oci.core.models.LaunchInstanceDetails(
            display_name=new_instance_name,
            compartment_id=my_instance_config[profile]['compartment-id'],
            availability_domain=ad_match[0].name,
            shape=my_instance_config[profile]['shape'],
            metadata=metadata,
            source_details=instance_source_via_image_details,
            create_vnic_details=create_vnic_details)
print(f'The instance details before launch:\n{launch_instance_details}\n')

compute_client_composite_operations = oci.core.ComputeClientCompositeOperations(my_compute_client)
response = compute_client_composite_operations.launch_instance_and_wait_for_state(
    launch_instance_details,
    wait_for_states=[oci.core.models.Instance.LIFECYCLE_STATE_RUNNING])
print('The instance is running')

instance = response.data
print(f'\nThe instance details now it is running:\n{instance}')

The instance details before launch:
{
  "agent_config": null,
  "availability_config": null,
  "availability_domain": "ruWb:EU-FRANKFURT-1-AD-1",
  "capacity_reservation_id": null,
  "compartment_id": "ocid1.compartment.oc1..xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "create_vnic_details": {
    "assign_private_dns_record": null,
    "assign_public_ip": null,
    "defined_tags": null,
    "display_name": null,
    "freeform_tags": null,
    "hostname_label": null,
    "nsg_ids": null,
    "private_ip": null,
    "skip_source_dest_check": null,
    "subnet_id": "ocid1.subnet.oc1.eu-frankfurt-1.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    "vlan_id": null
  },
  "dedicated_vm_host_id": null,
  "defined_tags": null,
  "display_name": "provisioning_with_python",
  "extended_metadata": null,
  "fault_domain": null,
  "freeform_tags": null,
  "hostname_label": null,
  "image_id": null,
  "instance_options": null,
  "ipxe_script": null,
  "is_pv_encr

### Getting the instance information

Now the instance is running you can get the information about your network so you can connect with your new instance. 

> Note: The first time the Oracle Solaris image boots it can take a bit of time before all the SMF services are initialized so you may need to wait a bit before connecting to it. 

It also shows how to get the information about all the instance you have in this compartment.

In [16]:
response = oci.pagination.list_call_get_all_results(my_compute_client.list_instances, my_instance_config[profile]['compartment-id'])
instances = []
for instance in response.data:
    if instance.lifecycle_state == 'TERMINATED':
        continue
    
    vnic_attachments = oci.pagination.list_call_get_all_results(my_compute_client.list_vnic_attachments, 
                                                                compartment_id=my_instance_config[profile]['compartment-id'], 
                                                                instance_id=instance.id).data

    if vnic_attachments:
        vnic = None
        for vnic_attachment in vnic_attachments:
            try:
                vnic = my_virtual_network_client.get_vnic(vnic_attachment.vnic_id).data
            except oci.exceptions.ServiceError:
                vnic = None
            if vnic and vnic.is_primary:
                break
    else:
        self._echo_error('Could not retrieve VNIC attachments')

    instances.append((
        instance.id,
        instance.display_name,
        instance.availability_domain[-4:],
        instance.time_created.strftime("%Y-%m-%d %H:%M:%S %Z"),
        instance.lifecycle_state.title(),
        vnic.private_ip if vnic else 'None',
        vnic.public_ip if vnic else 'None'
    ))

vnic = table = AsciiTable((('Private IP', vnic.private_ip), ('Public IP', vnic.public_ip)))
table.inner_heading_row_border = False
table.title = 'Instance provisioned'
print(table.table)

if instances:
    table = AsciiTable([('Name', 'AD', 'Time Created', 'State', 'Private IP', 'Public IP')] + sorted(instance[1:] for instance in instances))
    table.title = 'Compute Instances'
    print(table.table)
else:
    print('No instance found')

+Instance provisioned--------+
| Private IP | 10.0.0.219    |
| Public IP  | xxx.xx.xx.xxx |
+------------+---------------+
+Compute Instances-----------+------+-------------------------+---------+------------+----------------+
| Name                       | AD   | Time Created            | State   | Private IP | Public IP      |
+----------------------------+------+-------------------------+---------+------------+----------------+
| provisioning_with_python   | AD-1 | 2022-12-08 15:22:08 UTC | Running | 10.0.0.219 | xxx.xx.xx.xxx  |
+----------------------------+------+-------------------------+---------+------------+----------------+


## Other lifecycle actions

This is how you can do other lifecycle actions. Again it may take a while for the Oracle Solaris instance to be ready after the initial launch, so you may need to wait a bit if you are testing this.

In [17]:
instance_id = instance.id

Stopping the instance:

In [18]:
print(instance_id)

compute_client_composite_operations = oci.core.ComputeClientCompositeOperations(my_compute_client)
print('Waiting for Stopped state')
compute_client_composite_operations.instance_action_and_wait_for_state(
    instance_id=instance_id,
    action='SOFTSTOP',
    wait_for_states=[oci.core.models.Instance.LIFECYCLE_STATE_STOPPED],
    # waiter_kwargs={'wait_callback': self._wait_callback}
)
print('Done')

ocid1.instance.oc1.eu-frankfurt-1.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Waiting for Stopped state
Done


Starting the instance again:

In [19]:
compute_client_composite_operations = oci.core.ComputeClientCompositeOperations(my_compute_client)
print('Waiting for Running state')
compute_client_composite_operations.instance_action_and_wait_for_state(
    instance_id=instance_id,
    action='START',
    wait_for_states=[oci.core.models.Instance.LIFECYCLE_STATE_RUNNING],
)
print('Done')

Waiting for Running state
Done


Terminating the instance:

In [20]:
compute_client_composite_operations = oci.core.ComputeClientCompositeOperations(my_compute_client)
print('Waiting for termination')
compute_client_composite_operations.terminate_instance_and_wait_for_state(
    instance_id=instance_id,
    wait_for_states=[oci.core.models.Instance.LIFECYCLE_STATE_TERMINATED])
print('Done')

Waiting for termination
Done
